## Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import geocoder
from sklearn.cluster import KMeans
import folium

## Copy the url to a varible

In [ ]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

## Loading the html into DataFrame

In [ ]:
df=pd.read_html(url)

df=df[0]

df=df[df.Borough != 'Not assigned']

## DataFrame shape 

In [ ]:
df.shape

In [ ]:
clusters=len(df.Borough.unique())


In [ ]:
postal_df=pd.read_csv("https://cocl.us/Geospatial_data")

df=df.merge(postal_df,on='Postal Code')



In [ ]:

world_map=folium.Map([df.iloc[0,-2],df.iloc[0,-1]],zoom_start=10)



In [ ]:
from folium.plugins import MarkerCluster

marker_cluster=marker_cluster = MarkerCluster(name='1000 clustered icons',overlay=True,control=False,icon_create_function=None)

for x in zip(df.Latitude,df.Longitude,df.Borough):
    folium.CircleMarker([x[0],x[1]],popup=x[2],fill_color='green',fill_opacity=0.4).add_to(marker_cluster)
marker_cluster.add_to(world_map)

## Cluster_map of toronto

In [17]:
toronto_map=world_map
toronto_map

## Foursquare Stuff

In [25]:
clientid='5KSKODJSBN0AJGA3UVTHACSHOMRCD0UQTT3BG3DGY1S2ZFLC'
clientsecret='BAQHVX1GJBM0KHQZ0NRKOC4Q1412OCZMWGMDRMV3QHZECFLT'
version=20200628
radius=500
limit=100

## Function to get Required Features from Neighborhood Venues

In [123]:
features= ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

def get_features(df):
    
    
    if df.values.tolist()!=[]:
        df=df[features]
        
        df.columns=['venue_name','venue_category','venue_lat','venue_lng']
        df.loc[:,'venue_category']=df.loc[:,'venue_category'].map(lambda x : x[0]['name'])

        return pd.get_dummies(df.venue_category).sum().sort_values(ascending=False)[:2].index.tolist()

    elif df.values.tolist()==[]:
        return [0,0]
    else:
        return 1
    

## Funtion to get venues for each neighborhood

In [124]:
common_venues={}
def get_venues(latitude,longitude):
    
    ll=latitude,longitude

    url ='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(clientid,clientsecret,version,ll[0],ll[1],radius,limit)
    venue_df=pd.json_normalize(requests.get(url).json()['response']['groups'][0]['items'])
    
    
    return get_features(venue_df)


## Analysis and getting  Most Frequent Venues for each Neighborhood

In [125]:
df.loc[:,'First_common_category']=0
df.loc[:,'Second_common_category']=0

In [126]:
for x in range(len(df)):
    ll=df.Latitude[x],df.Longitude[x]
    url ='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(clientid,clientsecret,version,ll[0],ll[1],radius,limit)

    ls=get_features(pd.json_normalize(requests.get(url).json()['response']['groups'][0]['items']))
    
    if len(ls)==2:
        df.iloc[x,5]=ls[0]
        df.iloc[x,6]=ls[1]
    
   
    
    elif len(ls)==1:
        df.iloc[x,5]=ls[0]


    
    

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## KMeans Clustering

In [128]:
n_clusters=5
kmc=KMeans(n_clusters=n_clusters,random_state=0)

model=kmc.fit(pd.get_dummies(df))

In [129]:
df.loc[:,'Cluster_labels']=model.labels_

In [130]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [131]:
colors_array=cm.viridis(np.linspace(0,1,n_clusters))
colors=[colors.rgb2hex(x) for x in colors_array]

In [132]:
latlng_df=df.iloc[:,[3,4,7]]
col_df=pd.DataFrame([x for x in zip(range(5),colors)])
                     
map_df=pd.merge(latlng_df,col_df,left_on=latlng_df.loc[:,'Cluster_labels'],right_on=col_df.iloc[:,0])

map_df=map_df.rename({1:'color'},axis=1)

map_df=map_df.loc[:,['Latitude','Longitude','color']]

In [133]:
cluster_map=folium.Map([df.Latitude[0],df.Longitude[0]],zoom_start=10)

In [134]:
for x in zip(map_df.Latitude,map_df.Longitude,map_df.color,df.Cluster_labels):
    folium.CircleMarker([x[0],x[1]],color=x[2],fill_color=x[2],fill_opacity=0.7,popup=x[3]).add_to(cluster_map)

## Cluster Map of Toronto 

In [121]:
cluster_map